In [131]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#!conda install -c conda-forge geopy --yes 

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans





print('Libraries imported.')

Libraries imported.


In [132]:
!pip install folium  # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [7]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-86dbe5c8-c42b-4c0c-be7b-71a485ff1a0c',
    'IBM_API_KEY_ID': 'm68EwWLeHzirqJI0RKKD_WgQ1TqOiFZFV9nuvAucoTFq',
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET': 'toronto-donotdelete-pr-cougb4yv2dabqm',
    'FILE': 'Geospatial_Coordinates.csv'
}


In [16]:

body = client_f1e46e8fe1a34588b5e0301d7ec0275a.get_object(Bucket='toronto-donotdelete-pr-cougb4yv2dabqm',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

lat = pd.read_csv(body)
lat.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_f1e46e8fe1a34588b5e0301d7ec0275a = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='m68EwWLeHzirqJI0RKKD_WgQ1TqOiFZFV9nuvAucoTFq',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_f1e46e8fe1a34588b5e0301d7ec0275a.get_object(Bucket='toronto-donotdelete-pr-cougb4yv2dabqm',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()
body = client_f1e46e8fe1a34588b5e0301d7ec0275a.get_object(Bucket='toronto-donotdelete-pr-cougb4yv2dabqm',Key='capstone_3.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

loc = pd.read_csv(body)
loc.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [10]:
loc['Borough'].replace('Not assigned',np.nan,inplace=True)
loc['Neighbourhood'].replace('Not assigned',np.nan,inplace=True)

In [11]:
loc.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
tryloc=loc.dropna()

In [13]:
tryloc.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [14]:
tryloc.rename({'Postcode':'Postal Code'},axis=1,inplace=True)
tryloc.rename({'Neighbourhood':'Neighborhood'},axis=1,inplace=True)
tryloc.head(1)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods


In [18]:
det=tryloc.merge(lat)

In [19]:
address = 'Toronto city, Canada'

geolocator = Nominatim(user_agent="Canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of toronto city are 43.653963, -79.387207.


In [133]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(det['Latitude'], det['Longitude'], det['Borough'], det['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [21]:
CLIENT_ID = 'WR3OEM0VPYNFMSQYPKBJFPLCPYCNUHJPASR5QRBPTV5BX2P0' # your Foursquare ID
CLIENT_SECRET = 'TSYMU5G014VXJF2SBZOWBFE4KRHTOUQ0Y3HKVJKEUUCRSGGF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WR3OEM0VPYNFMSQYPKBJFPLCPYCNUHJPASR5QRBPTV5BX2P0
CLIENT_SECRET:TSYMU5G014VXJF2SBZOWBFE4KRHTOUQ0Y3HKVJKEUUCRSGGF


In [22]:
import io
import requests
url="https://cocl.us/Geospatial_data"
p=requests.get(url).text
df=pd.read_csv(io.StringIO(p))
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
neighborhood_latitude = det.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = det.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = det.loc[0, 'Neighborhood']

# type your answer here
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=WR3OEM0VPYNFMSQYPKBJFPLCPYCNUHJPASR5QRBPTV5BX2P0&client_secret=TSYMU5G014VXJF2SBZOWBFE4KRHTOUQ0Y3HKVJKEUUCRSGGF&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e61ce43a2e538001b402d4c'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# type your answer here

toronto_venues = getNearbyVenues(names=det['Neighborhood'],
                                   latitudes=det['Latitude'],
                                   longitudes=det['Longitude']
                                  )



Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design Exchange
Toro

In [40]:
print(toronto_venues.shape)
toronto_venues.head()

(1069, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
1,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
2,Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
3,Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
4,Harbourfront,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [41]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,25,25,25,25,25,25
Alderwood,3,3,3,3,3,3
Bathurst Manor,8,8,8,8,8,8
Bathurst Quay,1,1,1,1,1,1
Bayview Village,1,1,1,1,1,1
Bedford Park,11,11,11,11,11,11
Birch Cliff,1,1,1,1,1,1
Bloordale Gardens,4,4,4,4,4,4
Brockton,2,2,2,2,2,2


In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] =toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auto Workshop,Bakery,Bank,Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Creperie,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Flower Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,History Museum,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Mac & Cheese Joint,Market,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Movie Theater,Moving Target,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Transportation Service,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [138]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
df=toronto_grouped[['Restaurant','Neighborhood']]
toronto_make=pd.merge(df,det,on="Neighborhood")
toronto_make.head()


,Restaurant,Neighborhood,Postal Code,Borough,Latitude,Longitude
0,0.040,Adelaide,M5H,Downtown Toronto,43.650571,-79.384568
1,0.000,Alderwood,M8W,Etobicoke,43.602414,-79.543484
2,0.125,Bathurst Manor,M3H,North York,43.754328,-79.442259
3,0.000,Bathurst Quay,M5V,Downtown Toronto,43.628947,-79.394420
4,0.000,Bayview Village,M2K,North York,43.786947,-79.385975


In [49]:
det

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
6,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188


In [56]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                  venue  freq
0           Coffee Shop  0.08
1      Asian Restaurant  0.08
2            Steakhouse  0.08
3       Thai Restaurant  0.04
4  Colombian Restaurant  0.04


----Alderwood----
             venue  freq
0      Pizza Place  0.33
1         Pharmacy  0.33
2      Coffee Shop  0.33
3      Yoga Studio  0.00
4  Organic Grocery  0.00


----Bathurst Manor----
                 venue  freq
0  Fried Chicken Joint  0.12
1     Sushi Restaurant  0.12
2          Pizza Place  0.12
3           Restaurant  0.12
4          Coffee Shop  0.12


----Bathurst Quay----
                   venue  freq
0  Performing Arts Venue   1.0
1            Yoga Studio   0.0
2            Opera House   0.0
3    Moroccan Restaurant   0.0
4          Movie Theater   0.0


----Bayview Village----
            venue  freq
0  Shopping Plaza   1.0
1     Yoga Studio   0.0
2     Opera House   0.0
3   Movie Theater   0.0
4   Moving Target   0.0


----Bedford Park----
                venue  freq
0  

             venue  freq
0      Golf Course   1.0
1      Yoga Studio   0.0
2  Organic Grocery   0.0
3    Movie Theater   0.0
4    Moving Target   0.0


----Humewood-Cedarvale----
           venue  freq
0     Playground   0.5
1          Field   0.5
2    Opera House   0.0
3  Movie Theater   0.0
4  Moving Target   0.0


----India Bazaar----
               venue  freq
0  Fish & Chips Shop   0.5
1               Park   0.5
2        Yoga Studio   0.0
3        Opera House   0.0
4      Movie Theater   0.0


----Island airport----
                   venue  freq
0  Performing Arts Venue   1.0
1            Yoga Studio   0.0
2            Opera House   0.0
3    Moroccan Restaurant   0.0
4          Movie Theater   0.0


----Keelesdale----
             venue  freq
0   Discount Store   0.5
1   Sandwich Place   0.5
2      Yoga Studio   0.0
3  Organic Grocery   0.0
4    Moving Target   0.0


----Kensington Market----
                   venue  freq
0                   Café  0.11
1  Vietnamese Restaurant  

               venue  freq
0  Fish & Chips Shop   0.5
1               Park   0.5
2        Yoga Studio   0.0
3        Opera House   0.0
4      Movie Theater   0.0


----The Danforth West----
              venue  freq
0  Greek Restaurant   1.0
1       Yoga Studio   0.0
2   Organic Grocery   0.0
3     Movie Theater   0.0
4     Moving Target   0.0


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.17
1     Sandwich Place  0.17
2        Yoga Studio  0.08
3        Gas Station  0.08
4        Pizza Place  0.08


----Toronto Dominion Centre----
           venue  freq
0    Coffee Shop  0.22
1  Deli / Bodega  0.10
2     Restaurant  0.07
3            Gym  0.05
4    Salad Place  0.05


----Toronto Islands----
                       venue  freq
0                Coffee Shop  0.15
1  Middle Eastern Restaurant  0.05
2                   Tea Room  0.05
3               Dessert Shop  0.05
4                Pizza Place  0.05


----Trinity----
         venue  freq
0     Wine Bar  0.

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Asian Restaurant,Steakhouse,Colombian Restaurant,Sushi Restaurant,Concert Hall,Park,Pharmacy,Restaurant,Japanese Restaurant
1,Alderwood,Pharmacy,Coffee Shop,Pizza Place,Cuban Restaurant,Dumpling Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant
2,Bathurst Manor,Restaurant,Pizza Place,Video Store,Middle Eastern Restaurant,Deli / Bodega,Coffee Shop,Fried Chicken Joint,Sushi Restaurant,Diner,Ethiopian Restaurant
3,Bathurst Quay,Performing Arts Venue,Women's Store,Discount Store,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
4,Bayview Village,Shopping Plaza,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Diner


In [59]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 1, 1, 1, 1, 1, 1], dtype=int32)

In [60]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_make

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Restaurant,Neighborhood,Postal Code,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.040,Adelaide,M5H,Downtown Toronto,43.650571,-79.384568,1,Coffee Shop,Asian Restaurant,Steakhouse,Colombian Restaurant,Sushi Restaurant,Concert Hall,Park,Pharmacy,Restaurant,Japanese Restaurant
1,0.000,Alderwood,M8W,Etobicoke,43.602414,-79.543484,1,Pharmacy,Coffee Shop,Pizza Place,Cuban Restaurant,Dumpling Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant
2,0.125,Bathurst Manor,M3H,North York,43.754328,-79.442259,1,Restaurant,Pizza Place,Video Store,Middle Eastern Restaurant,Deli / Bodega,Coffee Shop,Fried Chicken Joint,Sushi Restaurant,Diner,Ethiopian Restaurant
3,0.000,Bathurst Quay,M5V,Downtown Toronto,43.628947,-79.394420,3,Performing Arts Venue,Women's Store,Discount Store,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
4,0.000,Bayview Village,M2K,North York,43.786947,-79.385975,1,Shopping Plaza,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Diner


In [134]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi,rest, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],toronto_merged['Restaurant'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Resturant '  + str(rest) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [62]:
toronto_merged['Cluster Labels'].isnull().value_counts()

False    109
Name: Cluster Labels, dtype: int64

In [44]:
toronto_merged['Cluster Labels'].replace(np.nan,3,inplace=True)

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
47,India Bazaar,-79.315572,0,Fish & Chips Shop,Park,Discount Store,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
61,Moore Park,-79.383160,0,Park,Moving Target,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
75,Rosedale,-79.377529,0,Park,Women's Store,Discount Store,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
88,Summerhill East,-79.383160,0,Park,Moving Target,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
91,The Beaches,-79.293031,0,Park,Trail,Other Great Outdoors,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store
92,The Beaches West,-79.315572,0,Fish & Chips Shop,Park,Discount Store,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
108,York Mills West,-79.400049,0,Park,Convenience Store,Women's Store,Dumpling Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store


In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,-79.384568,1,Coffee Shop,Asian Restaurant,Steakhouse,Colombian Restaurant,Sushi Restaurant,Concert Hall,Park,Pharmacy,Restaurant,Japanese Restaurant
1,Alderwood,-79.543484,1,Pharmacy,Coffee Shop,Pizza Place,Cuban Restaurant,Dumpling Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant
2,Bathurst Manor,-79.442259,1,Restaurant,Pizza Place,Video Store,Middle Eastern Restaurant,Deli / Bodega,Coffee Shop,Fried Chicken Joint,Sushi Restaurant,Diner,Ethiopian Restaurant
4,Bayview Village,-79.385975,1,Shopping Plaza,Women's Store,Discount Store,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant,Diner
5,Bedford Park,-79.419750,1,Italian Restaurant,Sushi Restaurant,Cupcake Shop,Fast Food Restaurant,Comfort Food Restaurant,Sandwich Place,Breakfast Spot,Coffee Shop,Ice Cream Shop,Juice Bar
6,Birch Cliff,-79.264848,1,Café,Women's Store,Dumpling Restaurant,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Discount Store
7,Bloordale Gardens,-79.577201,1,Convenience Store,Café,Liquor Store,Pizza Place,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
8,Brockton,-79.428191,1,Playground,Tech Startup,Flower Shop,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
9,Business Reply Mail Processing Centre 969 Eastern,-79.321558,1,Brewery,Auto Workshop,Women's Store,Electronics Store,Fish & Chips Shop,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant
11,Cabbagetown,-79.367675,1,Café,Pizza Place,Beer Store,Diner,Coffee Shop,Restaurant,Chinese Restaurant,Market,Italian Restaurant,Indian Restaurant


In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downsview,-79.490074,2,Bakery,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store
63,North Park,-79.490074,2,Bakery,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store
101,Upwood Park,-79.490074,2,Bakery,Women's Store,Dumpling Restaurant,Fish & Chips Shop,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Etobicoke,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,North York,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,North York,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,North York,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,North York,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,North York,3.0,Baseball Field,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Event Space
106,North York,3.0,Baseball Field,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Event Space
187,Scarborough,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,Etobicoke,3.0,Baseball Field,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Event Space
198,Etobicoke,3.0,Baseball Field,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Event Space


In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,Riverdale,-79.352188,4,Greek Restaurant,Women's Store,Discount Store,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant
93,The Danforth West,-79.352188,4,Greek Restaurant,Women's Store,Discount Store,Film Studio,Field,Fast Food Restaurant,Farmers Market,Ethiopian Restaurant,Electronics Store,Dumpling Restaurant


In [135]:
df2=toronto_merged[['Cluster Labels','Neighborhood']]

In [139]:
toronto_make=pd.merge(df2,toronto_make,on='Neighborhood')
toronto_make

,Cluster Labels,Neighborhood,Restaurant,Postal Code,Borough,Latitude,Longitude
0,3,Adelaide,0.040000,M5H,Downtown Toronto,43.650571,-79.384568
1,0,Alderwood,0.000000,M8W,Etobicoke,43.602414,-79.543484
2,4,Bathurst Manor,0.125000,M3H,North York,43.754328,-79.442259
3,0,Bathurst Quay,0.000000,M5V,Downtown Toronto,43.628947,-79.394420
4,0,Bayview Village,0.000000,M2K,North York,43.786947,-79.385975
5,0,Bedford Park,0.000000,M5M,North York,43.733283,-79.419750
6,0,Birch Cliff,0.000000,M1N,Scarborough,43.692657,-79.264848
7,0,Bloordale Gardens,0.000000,M9C,Etobicoke,43.643515,-79.577201
8,0,Brockton,0.000000,M6K,West Toronto,43.636847,-79.428191
9,0,Business Reply Mail Processing Centre 969 Eastern,0.000000,M7Y,East Toronto,43.662744,-79.321558
